# Zadanie domowe - łączenie danych!

Wykonujesz zlecenia dla firmy, która chce żebyśmy połączyli dane z trzech różnych źródeł. 

1. Pierwsze źródło danych to dane o klientach. Na szczęście zespół, który zajmuje się klientami, wie co robi i wszystkie dane dostępne są w bazie danych (`zad_domowe__clients.db` w tabeli `clients`).
2. Drugie źródło danych to dane o produktach. W tym przypadku dane dostępne są w pliku CSV (`zad_domowe__products.csv`).
3. Ostatnie źródło danych to dane o zamówieniach / fakturach. Tutaj niestety mamy problem. Okazuje się, że klient wszystkie faktury ma w postaci zrzutów z ekranu (pliki `zad_domowe__invoice_1.png`, `zad_domowe__invoice_2.png` etc.). 

Twoim zadaniem jest połączenie tych danych w jeden plik CSV, który będzie zawierał informacje o kliencie, produkcie i fakturze. Przygotuj plik CSV, który zawiera tylko niezbędne kolumny, nie ma żadnych kolumn powtórzonych. 

Jak rezultat wyciągania danych z faktur chcemy mieć w każdym wierszu informacje o zakupionym produkcie, ilości, kliencie, cenie jednostkowej.

Początek kodu znajdziesz w tym notebooku.

In [8]:
from pathlib import Path
import pandas as pd
import sqlite3
import base64
from datetime import date
import instructor
from pydantic import BaseModel
from IPython.display import Image
from openai import OpenAI
import os
from dotenv import load_dotenv

In [9]:
# będziemy potrzebować klucza API do OpenAI żeby odczytać dane z faktur

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_key)

## Czytamy dane klientów

In [10]:
with sqlite3.connect("zad_domowe__clients.db") as conn:
    clients_df = pd.read_sql_query("SELECT * FROM clients", conn)

clients_df.head()

,id,name,email,phone
0,1,Carla Ramsey,frypeter@example.net,(508)998-5166x12326
1,2,Howard Mcgee,jacobpatterson@example.org,001-669-353-6118x978
2,3,Eric Moore,lejacob@example.org,923-681-4646
3,4,Laura Beard,westdominique@example.org,(291)558-0966
4,5,Joshua Santiago,sheilamedina@example.com,+1-929-417-0571x74051


## Czytamy dane produktów

In [11]:
# TUTAJ WPISZ SWÓJ KOD
products_df = pd.read_csv("zad_domowe__products.csv", sep=";")

products_df.head()

,id,name,price
0,1,telewizor,85
1,2,laptop,47
2,3,smartfon,60
3,4,tablet,40
4,5,słuchawki,40


## Czytamy dane faktur

In [12]:
# na szczęście mamy już strukturę którą chcemy pobrać z faktur
class InvoiceInfoItem(BaseModel):
    description: str
    product_id: int
    quantity: int
    price: float


class InvoiceInfo(BaseModel):
    company_name: str
    customer_id: int
    customer_name: str
    invoice_number: int
    date: date
    # mamy tutaj do czynienia z listą elementów, więc danymi zagnieżdżonymi!
    items: list[InvoiceInfoItem]


RAW_DATA_PATH = Path("raw_inv")

instructor_openai_client = instructor.from_openai(OpenAI(api_key=openai_key))
orders = []
for image_path in sorted(RAW_DATA_PATH.glob("zad_domowe__invoice_*.png")):
    print(f"Processing {image_path}")
    with open(image_path, "rb") as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')

    # UWAGA! tutaj brakuje kodu!!! Co powinniśmy tutaj zrobić?
    invoice_info = instructor_openai_client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        response_model=InvoiceInfo,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Odczytaj informacje z faktury i zwróć je w podanym poniżej formacie InvoiceInfo."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{image_data}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ]
    )

    invoice_data = invoice_info.model_dump()
    for item in invoice_data["items"]:
        order = {
            "company_name": invoice_data["company_name"],
            "customer_id": invoice_data["customer_id"],
            "customer_name": invoice_data["customer_name"],
            "invoice_number": invoice_data["invoice_number"],
            "date": invoice_data["date"],
            "description": item["description"],
            "product_id": item["product_id"],
            "quantity": item["quantity"],
            "price": item["price"],
        }
        orders.append(order)

orders_df = pd.DataFrame(orders)

Processing raw_inv\zad_domowe__invoice_1.png
Processing raw_inv\zad_domowe__invoice_10.png
Processing raw_inv\zad_domowe__invoice_2.png
Processing raw_inv\zad_domowe__invoice_3.png
Processing raw_inv\zad_domowe__invoice_4.png
Processing raw_inv\zad_domowe__invoice_5.png
Processing raw_inv\zad_domowe__invoice_6.png
Processing raw_inv\zad_domowe__invoice_7.png
Processing raw_inv\zad_domowe__invoice_8.png
Processing raw_inv\zad_domowe__invoice_9.png


In [13]:
orders_df = pd.DataFrame(orders)
orders_df.head(10)

,company_name,customer_id,customer_name,invoice_number,date,description,product_id,quantity,price
0,Firma XYZ,3,Eric Moore,2,2024-05-25,router,7,3,48.0
1,Firma XYZ,3,Eric Moore,2,2024-05-25,narty,17,5,55.0
2,Firma XYZ,92,Valerie Stevens,11,2024-05-02,słuchawki,5,1,40.0
3,Firma XYZ,60,Troy Alexander,3,2024-05-19,ochraniacze,16,4,46.0
4,Firma XYZ,78,William Velazquez,4,2024-03-26,rolki,14,3,11.0
5,Firma XYZ,33,Steven Jackson,5,2024-07-23,pendrive (id produktu 10),10,4,47.0
6,Firma XYZ,33,Steven Jackson,5,2024-07-23,laptop (id produktu 2),2,2,47.0
7,Firma XYZ,8,Ronald Callahan,6,2024-03-14,słuchawki,5,4,40.0
8,Firma XYZ,8,Ronald Callahan,6,2024-03-14,ochraniacze,16,2,46.0
9,Firma XYZ,4,Laura Beard,7,2024-06-24,telewizor (id produktu 1),1,2,85.0


## Łączenie danych

In [14]:
# merge clients and orders
company_df = pd.merge(    
    clients_df,
    orders_df,
    left_on='id',
    right_on= 'customer_id',
    how='right')
company_df.head(100)

,id,name,email,phone,company_name,customer_id,customer_name,invoice_number,date,description,product_id,quantity,price
0,3,Eric Moore,lejacob@example.org,923-681-4646,Firma XYZ,3,Eric Moore,2,2024-05-25,router,7,3,48.0
1,3,Eric Moore,lejacob@example.org,923-681-4646,Firma XYZ,3,Eric Moore,2,2024-05-25,narty,17,5,55.0
2,92,Valerie Stevens,xlowe@example.com,757-943-1579x98346,Firma XYZ,92,Valerie Stevens,11,2024-05-02,słuchawki,5,1,40.0
3,60,Troy Alexander,nturner@example.net,+1-212-349-3992x430,Firma XYZ,60,Troy Alexander,3,2024-05-19,ochraniacze,16,4,46.0
4,78,William Velazquez,lisa25@example.net,(765)602-1572x949,Firma XYZ,78,William Velazquez,4,2024-03-26,rolki,14,3,11.0
5,33,Steven Jackson,jeremy07@example.com,481.258.2434,Firma XYZ,33,Steven Jackson,5,2024-07-23,pendrive (id produktu 10),10,4,47.0
6,33,Steven Jackson,jeremy07@example.com,481.258.2434,Firma XYZ,33,Steven Jackson,5,2024-07-23,laptop (id produktu 2),2,2,47.0
7,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,Firma XYZ,8,Ronald Callahan,6,2024-03-14,słuchawki,5,4,40.0
8,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,Firma XYZ,8,Ronald Callahan,6,2024-03-14,ochraniacze,16,2,46.0
9,4,Laura Beard,westdominique@example.org,(291)558-0966,Firma XYZ,4,Laura Beard,7,2024-06-24,telewizor (id produktu 1),1,2,85.0


In [15]:
# merge clients&orders and products
company_df = pd.merge(
    company_df,
    products_df,
    left_on='product_id',
    right_on='id',
    how='left')
company_df.head(100).sort_values(by='id_x')

,id_x,name_x,email,phone,company_name,customer_id,customer_name,invoice_number,date,description,product_id,quantity,price_x,id_y,name_y,price_y
0,3,Eric Moore,lejacob@example.org,923-681-4646,Firma XYZ,3,Eric Moore,2,2024-05-25,router,7,3,48.0,7,router,48
1,3,Eric Moore,lejacob@example.org,923-681-4646,Firma XYZ,3,Eric Moore,2,2024-05-25,narty,17,5,55.0,17,narty,55
9,4,Laura Beard,westdominique@example.org,(291)558-0966,Firma XYZ,4,Laura Beard,7,2024-06-24,telewizor (id produktu 1),1,2,85.0,1,telewizor,85
7,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,Firma XYZ,8,Ronald Callahan,6,2024-03-14,słuchawki,5,4,40.0,5,słuchawki,40
8,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,Firma XYZ,8,Ronald Callahan,6,2024-03-14,ochraniacze,16,2,46.0,16,ochraniacze,46
5,33,Steven Jackson,jeremy07@example.com,481.258.2434,Firma XYZ,33,Steven Jackson,5,2024-07-23,pendrive (id produktu 10),10,4,47.0,10,pendrive,47
6,33,Steven Jackson,jeremy07@example.com,481.258.2434,Firma XYZ,33,Steven Jackson,5,2024-07-23,laptop (id produktu 2),2,2,47.0,2,laptop,47
14,44,Kimberly Marquez,joseph28@example.net,+1-688-249-5941x6260,Firma XYZ,44,Kimberly Marquez,10,2024-03-14,laptop,2,1,47.0,2,laptop,47
15,44,Kimberly Marquez,joseph28@example.net,+1-688-249-5941x6260,Firma XYZ,44,Kimberly Marquez,10,2024-03-14,klawiatura,9,2,65.0,9,klawiatura,65
3,60,Troy Alexander,nturner@example.net,+1-212-349-3992x430,Firma XYZ,60,Troy Alexander,3,2024-05-19,ochraniacze,16,4,46.0,16,ochraniacze,46


In [16]:
company_df = (
    company_df
    .drop(columns=["customer_id", "customer_name", "id_y", 'price_y','description'])
    .rename(columns={
        'id_x' : 'ID klienta',
        'name_x': 'Nazwa klienta',
        'company_name' : 'Nazwa firmy',
        'invoice_number' : 'Numer faktury',
        'date' : 'Data faktury',
        'description' : 'Opis',
        'product_id' : 'ID produktu',
        'quantity' : 'Ilość',
        'price_x' : 'Cena',
        'email' : 'Email',
        'phone' : 'Telefon',
        'name_y' : 'Nazwa produktu',
    })
)
company_df.head(100).sort_values(by='ID klienta')

,ID klienta,Nazwa klienta,Email,Telefon,Nazwa firmy,Numer faktury,Data faktury,ID produktu,Ilość,Cena,Nazwa produktu
0,3,Eric Moore,lejacob@example.org,923-681-4646,Firma XYZ,2,2024-05-25,7,3,48.0,router
1,3,Eric Moore,lejacob@example.org,923-681-4646,Firma XYZ,2,2024-05-25,17,5,55.0,narty
9,4,Laura Beard,westdominique@example.org,(291)558-0966,Firma XYZ,7,2024-06-24,1,2,85.0,telewizor
7,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,Firma XYZ,6,2024-03-14,5,4,40.0,słuchawki
8,8,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085,Firma XYZ,6,2024-03-14,16,2,46.0,ochraniacze
5,33,Steven Jackson,jeremy07@example.com,481.258.2434,Firma XYZ,5,2024-07-23,10,4,47.0,pendrive
6,33,Steven Jackson,jeremy07@example.com,481.258.2434,Firma XYZ,5,2024-07-23,2,2,47.0,laptop
14,44,Kimberly Marquez,joseph28@example.net,+1-688-249-5941x6260,Firma XYZ,10,2024-03-14,2,1,47.0,laptop
15,44,Kimberly Marquez,joseph28@example.net,+1-688-249-5941x6260,Firma XYZ,10,2024-03-14,9,2,65.0,klawiatura
3,60,Troy Alexander,nturner@example.net,+1-212-349-3992x430,Firma XYZ,3,2024-05-19,16,4,46.0,ochraniacze


In [21]:
company_df = company_df[['ID klienta', 'Nazwa klienta', 'Email', 'Telefon', 'Nazwa firmy', 'Numer faktury', 'Data faktury', 'ID produktu','Nazwa produktu', 'Ilość', 'Cena']]

# company_df = company_df.sort_values(by='ID klienta')

company_df.head(100)
company_df.to_csv('zad_domowe__orders.csv', sep=';', index=True)